In [39]:
#uses DotProduct_Systolic - v1.1x
#uses RegisteredMultiplier  - v1.0x
#uses AdderTree - v1.2x

In [40]:
import sys
import math

In [41]:
#N>=2
N = input('Enter N: ')

Enter N: 30


In [42]:
N = int(N)
if(N<2):
    N=2

In [43]:
#MAMCS>=2
MAMCS = input('Enter Multiply-Add Max Component Size (>=2, <= {}, empty for 2): '.format(N-1))

Enter Multiply-Add Max Component Size (>=2, <= 29, empty for 2): 12


In [44]:
#AdderSize>=2
AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')

Enter maximum size allowed for base adders (empty for 2): 3


In [45]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH Default Value (empty for 10): ')

Enter IN_WIDTH Default Value (empty for 10): 


In [46]:
#INPUT_REG_DEPTH>=0
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH Default Value (empty for 1): ')

Enter INPUT_REG_DEPTH Default Value (empty for 1): 


In [47]:
#MULT_PIPE_DEPTH>=0
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH Default Value (empty for 1): ')

Enter MULT_PIPE_DEPTH Default Value (empty for 1): 


In [48]:
if (not MAMCS):
    MAMCS = N-1
else:
    MAMCS = int(MAMCS)
if MAMCS>=(N-1):
    MAMCS = N-1
    NumOfMaxSizeComponents=1
    NonMaxSizeComponentSize=1;
else:
    if(MAMCS<2):
        MAMCS=2
    NumOfMaxSizeComponents = math.floor(N/MAMCS)
    if (N/MAMCS) == NumOfMaxSizeComponents:
        NonMaxSizeComponentSize=0
    else:
        NonMaxSizeComponentSize = N - MAMCS*NumOfMaxSizeComponents
NumOfComponents = NumOfMaxSizeComponents + (NonMaxSizeComponentSize!=0)

if not AdderSize:
    AdderSize=2
else:
    AdderSize = int(AdderSize)
    if AdderSize<2:
        AdderSize=2
    if AdderSize>NumOfComponents:
        AdderSize=NumOfComponents

if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if(IN_WIDTH<1):
        IN_WIDTH=1

if not INPUT_REG_DEPTH:
    INPUT_REG_DEPTH=1
else:
    INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
    if(INPUT_REG_DEPTH<0):
        INPUT_REG_DEPTH=0

if not MULT_PIPE_DEPTH:
    MULT_PIPE_DEPTH=1
else:
    MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
    if(MULT_PIPE_DEPTH<0):
        MULT_PIPE_DEPTH=0

In [49]:
NonMaxSizeComponent_Mult_Register_Before = 1; #1: before 0:after
NonMaxSizeComponent_Size2up_Register_Before = 0; #1: before 0:after

In [ ]:
ModuleName = "DotProduct_{}_noHR_C{}_A{}".format(N, MAMCS, AdderSize)

In [50]:
__Print_To_File = 0

if __Print_To_File<=0:
    of=sys.stdout
else:
    of=open("./"+ModuleName+".v".format(N, MAMCS, AdderSize), 'w+')

In [51]:
print("`timescale 1ns / 1ps\n", file=of)
print("module "+ModuleName, file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
print("input inReady,", file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("A{}, ".format(i), end='', file=of)
print(file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("B{}, ".format(i), end='', file=of)
print(file=of)
print("output outReady,", file=of)
lgn=math.ceil(math.log2(N)) #>=1
al=-1+lgn #>=0
if al==0:
    print("output signed [2*IN_WIDTH:0] DP,", file=of)
else:
    print("output signed [(2*IN_WIDTH)+{}:0] DP,".format(al), file=of)
print("output earlyOutReady", file=of)
print(");\n", file=of)

`timescale 1ns / 1ps

module DotProduct_30_noHR_C12_A3
#(
parameter IN_WIDTH = 10,
parameter INPUT_REG_DEPTH = 1,
parameter MULT_PIPE_DEPTH = 1
)(
input clk, reset, enable,
input inReady,
input signed [IN_WIDTH-1:0] A0, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, A14, A15, A16, A17, A18, A19, A20, A21, A22, A23, A24, A25, A26, A27, A28, A29, 
input signed [IN_WIDTH-1:0] B0, B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, B12, B13, B14, B15, B16, B17, B18, B19, B20, B21, B22, B23, B24, B25, B26, B27, B28, B29, 
output outReady,
output signed [(2*IN_WIDTH)+4:0] DP,
output earlyOutReady
);



In [52]:
lgMAMCS=math.ceil(math.log2(MAMCS)) #>=1
palM=lgMAMCS-1 #>=0
print("wire DPOutReady;", file=of)
for i in range (NumOfMaxSizeComponents):
    if palM==0:
        print("wire signed [(2*IN_WIDTH):0] DPpart{};".format(i), file=of)
    else:
        print("wire signed [(2*IN_WIDTH)+{}:0] DPpart{};".format(palM, i), file=of)
    print("DotProduct_Systolic_{}#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )".format(MAMCS), file=of)
    print("\tDPS{}_{}(clk, reset, enable,".format(MAMCS, i), file=of)
    print("\tinReady,", file=of)
    print("\t", end='', file=of)
    for j in range(MAMCS):
        print("A{}, ".format(MAMCS*i+j), end='', file=of)
    print(file=of)
    print("\t", end='', file=of)
    for j in range(MAMCS):
        print("B{}, ".format(MAMCS*i+j), end='', file=of)
    print(file=of)
    if i==0:
        print("\tDPOutReady,", file=of)
    else:
        print("\tDPOutReady{}, //not used".format(i), file=of)
    print("\tDPpart{},".format(i), file=of)
    print("\tDPEarlyOutReady{}); //not used\n".format(i), file=of)

wire DPOutReady;
wire signed [(2*IN_WIDTH)+3:0] DPpart0;
DotProduct_Systolic_12#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )
	DPS12_0(clk, reset, enable,
	inReady,
	A0, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, 
	B0, B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, 
	DPOutReady,
	DPpart0,
	DPEarlyOutReady0); //not used

wire signed [(2*IN_WIDTH)+3:0] DPpart1;
DotProduct_Systolic_12#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )
	DPS12_1(clk, reset, enable,
	inReady,
	A12, A13, A14, A15, A16, A17, A18, A19, A20, A21, A22, A23, 
	B12, B13, B14, B15, B16, B17, B18, B19, B20, B21, B22, B23, 
	DPOutReady1, //not used
	DPpart1,
	DPEarlyOutReady1); //not used



In [53]:
if NonMaxSizeComponentSize==1:
    palN = -1;
    print("wire signed [(2*IN_WIDTH)-1:0] DPpart{};".format(NumOfMaxSizeComponents), file=of)
    print("RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),", file=of)
    print(".INPUT_REG_DEPTH(INPUT_REG_DEPTH", end='', file=of)
    if NonMaxSizeComponent_Mult_Register_Before>0:
        print("+{}".format(MAMCS-1), end='', file=of)
    print("),", file=of)
    print(".MULT_PIPE_DEPTH(MULT_PIPE_DEPTH", end='', file=of)
    if NonMaxSizeComponent_Mult_Register_Before<=0:
        #has OR, currently not SRL optimized, can use method for NonMaxSizeComponentSize>1
        print("+{}".format(MAMCS-1), end='', file=of)
    print("))", file=of)
    print("M{}(clk, reset, enable,".format(NumOfMaxSizeComponents), file=of)
    print("inReady,", file=of)
    print("A{}, B{},".format(MAMCS*NumOfMaxSizeComponents, MAMCS*NumOfMaxSizeComponents), file=of)
    print("MOR{}, //not used".format(NumOfMaxSizeComponents), file=of)
    print("DPpart{},".format(NumOfMaxSizeComponents), file=of)
    print("eMOR{}); //not used".format(NumOfMaxSizeComponents), file=of)
    print(file=of)

In [54]:
if NonMaxSizeComponentSize>1:
    LGNonMaxSizeComponentSize=math.ceil(math.log2(NonMaxSizeComponentSize)) #>=1
    palN=LGNonMaxSizeComponentSize-1 #>=0
    if palN==0:
        print("wire signed [2*IN_WIDTH:0] DPpart{};".format(NumOfMaxSizeComponents), file=of)
    else:
        print("wire signed [(2*IN_WIDTH)+{}:0] DPpart{};".format(palN, NumOfMaxSizeComponents), file=of)
    arn = MAMCS-NonMaxSizeComponentSize #>=1
    print("DotProduct_Systolic_{}#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH".format(NonMaxSizeComponentSize), end='', file=of)
    if NonMaxSizeComponent_Size2up_Register_Before>0:
        print("+{}".format(arn), end='', file=of)
    print("), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )", file=of)
    print("\tDPS{}(clk, reset, enable,".format(NonMaxSizeComponentSize), file=of)
    print("\tinReady,", file=of)
    print("\t", end='', file=of)
    for j in range(NonMaxSizeComponentSize):
        print("A{}, ".format(MAMCS*NumOfMaxSizeComponents+j), end='', file=of)
    print(file=of)
    print("\t", end='', file=of)
    for j in range(NonMaxSizeComponentSize):
        print("B{}, ".format(MAMCS*NumOfMaxSizeComponents+j), end='', file=of)
    print(file=of)
    print("\tDPOutReady{}, //not used".format(NumOfMaxSizeComponents), file=of)
    print("\tDPpart{},".format(NumOfMaxSizeComponents), file=of)
    print("\tDPEarlyOutReady{}); //not used\n".format(NumOfMaxSizeComponents), file=of)
    
    if NonMaxSizeComponent_Size2up_Register_Before<=0:
        #no OR, SRL optimized
        if palN==0:
            print("reg signed [2*IN_WIDTH:0] DPpart{}R [1:{}];".format(NumOfMaxSizeComponents, arn), file=of)
        else:
            print("reg signed [(2*IN_WIDTH)+{}:0] DPpart{}R [1:{}];".format(palN, NumOfMaxSizeComponents, arn), file=of)
        if arn>2:
            print("integer i;", file=of)
        print("always @(posedge clk) begin", file=of)
        print("\tif(reset) begin", file=of)
        print("\t\t//Do Nothing", file=of)
        print("\tend", file=of)
        print("\telse if(enable) begin", file=of)
        print("\t\tDPpart{}R[1] <= DPpart{};".format(NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)
        if arn==2:
            print("\t\tDPpart{}R[2] <= DPpart{}R[1];".format(NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)
        elif arn>2:
            print("\t\tfor(i=1;i<{};i=i+1) begin".format(arn), file=of)
            print("\t\t\tDPpart{}R[i+1] <= DPpart{}R[i];".format(NumOfMaxSizeComponents, NumOfMaxSizeComponents), file=of)
            print("\t\tend", file=of)
        print("\tend", file=of)
        print("end", file=of)

    print(file=of)

wire signed [(2*IN_WIDTH)+2:0] DPpart2;
DotProduct_Systolic_6#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )
	DPS6(clk, reset, enable,
	inReady,
	A24, A25, A26, A27, A28, A29, 
	B24, B25, B26, B27, B28, B29, 
	DPOutReady2, //not used
	DPpart2,
	DPEarlyOutReady2); //not used

reg signed [(2*IN_WIDTH)+2:0] DPpart2R [1:6];
integer i;
always @(posedge clk) begin
	if(reset) begin
		//Do Nothing
	end
	else if(enable) begin
		DPpart2R[1] <= DPpart2;
		for(i=1;i<6;i=i+1) begin
			DPpart2R[i+1] <= DPpart2R[i];
		end
	end
end



In [55]:
if NonMaxSizeComponentSize>=1:
    PalD = palM-palN #>=0
else:
    PalD = 0

if PalD>0: #sign extention needed
    if palM==0:
        print("wire signed [(2*IN_WIDTH):0] DPpart{}E = ".format(NumOfMaxSizeComponents), end='', file=of)    
    else:
        print("wire signed [(2*IN_WIDTH)+{}:0] DPpart{}E = ".format(palM, NumOfMaxSizeComponents), end='', file=of)
    print("{{{{{}{{DPpart{}".format(PalD, NumOfMaxSizeComponents), end='', file=of) 
    if NonMaxSizeComponentSize>1 and NonMaxSizeComponent_Size2up_Register_Before<=0:
        print("R[{}]".format(arn), end='', file=of)
    if palN==-1:
        print("[(2*IN_WIDTH)-1]}}", end='', file=of)                
    elif palN==0:
        print("[(2*IN_WIDTH)]}}", end='', file=of)    
    else:
        print("[(2*IN_WIDTH)+{}]}}}}".format(palN), end='', file=of)    
    print(",DPpart{}".format(NumOfMaxSizeComponents), end='', file=of)
    if NonMaxSizeComponentSize>1 and NonMaxSizeComponent_Size2up_Register_Before<=0:
        print("R[{}]".format(arn), end='', file=of)
    print("};", file=of)
    print(file=of)

wire signed [(2*IN_WIDTH)+3:0] DPpart2E = {{1{DPpart2R[6][(2*IN_WIDTH)+2]}},DPpart2R[6]};



In [56]:
print("AdderTree_{}_A{}#( .IN_WIDTH((2*IN_WIDTH)+{}) )".format(NumOfComponents, AdderSize, lgMAMCS), file=of)
print("\tAT(clk, reset, enable, DPOutReady,", file=of)
print("\t", end='', file=of)
for i in range(NumOfMaxSizeComponents):
    print("DPpart{}, ".format(i), end='', file=of)
if NonMaxSizeComponentSize!=0:
    print("DPpart{}".format(NumOfMaxSizeComponents), end='', file=of)
    if PalD>0:
        print("E", end='', file=of)
    elif NonMaxSizeComponentSize>1 and NonMaxSizeComponent_Size2up_Register_Before<=0:
        print("R[{}]".format(arn), end='', file=of)
    print(",", end='', file=of)
print("\n\toutReady, DP, earlyOutReady);\n", file=of)

AdderTree_3_A3#( .IN_WIDTH((2*IN_WIDTH)+4) )
	AT(clk, reset, enable, DPOutReady,
	DPpart0, DPpart1, DPpart2E,
	outReady, DP, earlyOutReady);



In [57]:
print("endmodule", file=of)

endmodule


In [58]:
if __Print_To_File>0:
    of.close()